In [9]:
import numpy as np
class Page_Managment():
    def __init__(self, max_page=10, max_cached_page=3):
        self.max_page = max_page
        self.max_cached_page = max_cached_page
    def genReqs(self, req_cnt=10):
        self.reqs = np.random.randint(self.max_page, size=(req_cnt))
    def resetCache(self):
        self.cache = []
        self.idx = 0
        self.miss_times = 0
        self.out = []
    def getOut(self):
        return ', '.join(self.out)
    def LRU_step(self):
        if self.idx >= len(self.reqs): # oob
            return
        
        # match
        for idx, i in enumerate(self.cache):
            if i['page_num'] == self.reqs[self.idx]:
                self.cache[idx]['idx'] = self.idx # update LRU
                self.idx += 1
                return
        
        # drop LRU
        if len(self.cache) >= self.max_cached_page:
            self.cache = sorted(self.cache, key=lambda x: x['idx'])
            self.out.append(str(self.cache[0]['page_num']))
            self.cache = self.cache[1:]
            
        self.cache.append({
            'idx': self.idx,
            'page_num': self.reqs[self.idx]
         })
        self.idx += 1
        self.miss_times += 1

    def FIFO_step(self):
        if self.idx >= len(self.reqs): # oob
            return
        
        # match
        for idx, i in enumerate(self.cache):
            if i['page_num'] == self.reqs[self.idx]:
                self.idx += 1
                return
        
        # drop LRU
        if len(self.cache) >= self.max_cached_page:
            self.cache = sorted(self.cache, key=lambda x: x['idx'])
            self.out.append(str(self.cache[0]['page_num']))
            self.cache = self.cache[1:]
            
        self.cache.append({
            'idx': self.idx,
            'page_num': self.reqs[self.idx]
         })
        self.idx += 1
        self.miss_times += 1

In [ ]:
import bimpy
import numpy as np

ctx = bimpy.Context()
ctx.init(800, 530, "EXP03")
with ctx:
        bimpy.themes.set_light_theme()
COLORS = bimpy.Int(10)
REQ_LEN = bimpy.Int(10)
CACHE = bimpy.Int(3)

colors = [0x4b19e6, 0x4bb43c, 0x19e1ff, 0xc88200, 0x3182f5, 0xb41e91, 0xf0f046, 0xf032e6, 0xd2f53c,
                  0xfabebe, 0x008080, 0xe6beff, 0xaa6e28, 0xfffac8, 0x800000, 0xaaffc3, 0x808000, 0xffd8b1,
                  0x000080, 0x808080, 0xFFFFFF, 0x000000]

page = Page_Managment(max_page=COLORS.value, max_cached_page=CACHE.value)
page.genReqs(REQ_LEN.value)
page.resetCache()
    
while not ctx.should_close():
        ctx.new_frame()

        bimpy.set_next_window_pos(bimpy.Vec2(20, 20), bimpy.Condition.Once)
        bimpy.set_next_window_size(bimpy.Vec2(600, 200), bimpy.Condition.Once)
        bimpy.begin("Pages")
        
        bimpy.text("Page Num Color")
        window_pos = bimpy.get_window_pos() + bimpy.Vec2(70, 50)
        
        cnt = 0
        for idx1 in range(3):
            for idx2 in range(10):
                if cnt < COLORS.value:
                    point = bimpy.Vec2(idx2*30, idx1*30)
                    bimpy.add_circle_filled(window_pos + point, 8, 0xAF000000 + colors[cnt], 100)
                cnt += 1
        
        
        for _ in range(COLORS.value // 10 * 2 + 1):
            bimpy.text(" ")
   
        bimpy.text("Page Usage")
    
        # cache points
        for idx in range(CACHE.value):
            point = bimpy.Vec2(idx*30, (COLORS.value // 10 + 1) * 35 + 20)
            if idx >= len(page.cache):
                bimpy.add_circle_filled(window_pos + point, 8, 0xAF000000, 100)
            else:
                cnt = page.cache[idx]['page_num']
                bimpy.add_circle_filled(window_pos + point, 8, 0xAF000000 + colors[cnt], 100)
        
        for _ in range(3):
            bimpy.text(" ")
            
        bimpy.text("Page Removed from Cache")
        bimpy.text(page.getOut())
        
        bimpy.end()

        bimpy.set_next_window_pos(bimpy.Vec2(20, 230), bimpy.Condition.Once)
        bimpy.set_next_window_size(bimpy.Vec2(600, 220), bimpy.Condition.Once)
        bimpy.begin("Controls")
        
        bimpy.text("Experiment3  Chengxuan Ying 201785071")
        bimpy.input_int("Page Counts (<={})".format(len(colors)), COLORS)
        bimpy.input_int("Cache Counts (<={})".format(len(colors)), CACHE)
        bimpy.input_int("Requests Counts", REQ_LEN)
        
        if bimpy.button("Generate Requests"):
            page = Page_Managment(max_page=COLORS.value, max_cached_page=CACHE.value)
            page.genReqs(REQ_LEN.value)
            page.resetCache()
        
        bimpy.text("One Click One Step")
        bimpy.same_line()
        if bimpy.button("LRU"):
            page.LRU_step()
        bimpy.same_line()
        if bimpy.button("FIFO"):
            page.FIFO_step()
            
        bimpy.text("One Click Five Step")
        bimpy.same_line()
        if bimpy.button("LRUx5"):
            for _ in range(5):
                page.LRU_step()
                
        bimpy.same_line()
        if bimpy.button("FIFOx5"):
            for _ in range(5):
                page.FIFO_step()
            
        bimpy.text("Goto End")
        bimpy.same_line()
        if bimpy.button("LRUx99999"):
            for _ in range(99999):
                page.LRU_step()
                
        bimpy.same_line()
        if bimpy.button("FIFOx99999"):
            for _ in range(99999):
                page.FIFO_step()
        
        bimpy.text('Miss times: ' +  str(page.miss_times) )
        bimpy.same_line()
        bimpy.text( 'Miss rate: '+ str(page.miss_times/len(page.reqs)*100)  + '%')

        bimpy.end()
        ctx.render()